In [26]:
import sympy
from sympy.physics import vector
from IPython.display import display

sympy.init_printing(use_latex="mathjax")

In [27]:
N = vector.ReferenceFrame("N")
A, m, k, t, t_0, sigma_0 = sympy.symbols("A m k t t_0 \\sigma_0")
v_x, v_y, vx_0, vy_0, dv = sympy.symbols("v_x v_y v_{x_0} v_{y_0} {\\Delta}v")
M1_x, M1_y, M2 = sympy.symbols("M_{1_x} M_{1_y} M_2")
v = v_x*N.x + v_y*N.y
v_0 = vx_0*N.x + vy_0*N.y
M1 = M1_x*N.x + M1_y*N.y
sigma = A*sympy.exp(-m*vector.dot(v - v_0, v - v_0)/(2*k*(t - t_0)))

def eval_at_zero(expression):
    return expression.subs(vx_0, 0.0).subs(vy_0, 0.0).subs(t, 0.0)

def taylor(expression, variables = [t, vx_0, vy_0]):
    expansion = sympy.simplify(eval_at_zero(expression)/eval_at_zero(expression))
    for var in variables:
        expansion = expansion + sympy.simplify(eval_at_zero(sympy.diff(expression, var))*var/eval_at_zero(expression))
        
#    for var1 in variables:
#        for var2 in variables:
#            expansion = expansion + sympy.simplify(eval_at_zero(sympy.diff(sympy.diff(expression, var1), var2))*var1*var2/(2*eval_at_zero(expression)))
    return expansion*eval_at_zero(expression)

def zeroeth_moments(expression, vrange=[-dv, 0.0, dv]):
    moment = m -m
    for vx in vrange:
        for vy in vrange:
            moment = moment + (expression).subs(v_x, vx).subs(v_y, vy)
    return sympy.simplify(moment)

    
def first_moments(expression, vrange=[-dv, 0.0, dv]):
    moment = v - v
    for vx in vrange:
        for vy in vrange:
            moment = moment + (v*expression).subs(v_x, vx).subs(v_y, vy)
    return moment.simplify()

    
def second_moments(expression, vrange=[-dv, 0.0, dv]):
    moment = m - m
    for vx in vrange:
        for vy in vrange:
            moment = moment + (vector.dot(v, v)*expression).subs(v_x, vx).subs(v_y, vy)
    return sympy.simplify(moment)

In [28]:
linearization = taylor(sigma)
linearization

                                                           ⎛  2      2⎞
                                                         m⋅⎝vₓ  + v_y ⎠
  ⎛        ⎛  2      2⎞                               ⎞  ──────────────
  ⎜    m⋅t⋅⎝vₓ  + v_y ⎠   m⋅vₓ⋅v_{x_0}   m⋅v_y⋅v_{y_0}⎟      2⋅k⋅t₀    
A⋅⎜1 + ──────────────── - ──────────── - ─────────────⎟⋅ℯ              
  ⎜              2            k⋅t₀            k⋅t₀    ⎟                
  ⎝        2⋅k⋅t₀                                     ⎠                

In [29]:
zmoment = zeroeth_moments(linearization) - 1
zmoment

                                                                           2  
                2                   2                           m⋅{\Delta}v   
     m⋅{\Delta}v         m⋅{\Delta}v                            ────────────  
     ────────────        ────────────                        2     2⋅k⋅t₀     
        2⋅k⋅t₀               k⋅t₀           2⋅A⋅m⋅t⋅{\Delta}v ⋅ℯ              
4⋅A⋅ℯ             + 4⋅A⋅ℯ             + A + ──────────────────────────────── +
                                                             2                
                                                         k⋅t₀                 

                                2    
                     m⋅{\Delta}v     
                     ────────────    
                  2      k⋅t₀        
 4⋅A⋅m⋅t⋅{\Delta}v ⋅ℯ                
 ──────────────────────────────── - 1
                  2                  
              k⋅t₀                   

In [30]:
fmoment = first_moments(linearization) - M1
fmoment

⎛                           ⎛            2                 2⎞          ⎞
⎜                           ⎜ m⋅{\Delta}v       m⋅{\Delta}v ⎟          ⎟
⎜                           ⎜ ────────────      ────────────⎟          ⎟
⎜                         2 ⎜    2⋅k⋅t₀             k⋅t₀    ⎟          ⎟
⎜  2⋅A⋅m⋅v_{x_0}⋅{\Delta}v ⋅⎝ℯ             + 2⋅ℯ            ⎠          ⎟
⎜- ────────────────────────────────────────────────────────── - M_{1_x}⎟
⎝                             k⋅t₀                                     ⎠ n_x + ⎛                           ⎛            2                 2⎞          ⎞
⎜                           ⎜ m⋅{\Delta}v       m⋅{\Delta}v ⎟          ⎟
⎜                           ⎜ ────────────      ────────────⎟          ⎟
⎜                         2 ⎜    2⋅k⋅t₀             k⋅t₀    ⎟          ⎟
⎜  2⋅A⋅m⋅v_{y_0}⋅{\Delta}v ⋅⎝ℯ             + 2⋅ℯ            ⎠          ⎟
⎜- ────────────────────────────────────────────────────────── - M_{1_y}⎟
⎝                             k⋅t₀           

In [31]:
smoment = second_moments(linearization) - M2
smoment

               ⎛                    2                       2                 
               ⎜         m⋅{\Delta}v             m⋅{\Delta}v                  
               ⎜         ────────────            ────────────                 
             2 ⎜      2     2⋅k⋅t₀            2      k⋅t₀                    2
2⋅A⋅{\Delta}v ⋅⎝2⋅k⋅t₀ ⋅ℯ             + 4⋅k⋅t₀ ⋅ℯ             + m⋅t⋅{\Delta}v 
──────────────────────────────────────────────────────────────────────────────
                                                                2             
                                                            k⋅t₀              

             2                                2⎞     
  m⋅{\Delta}v                      m⋅{\Delta}v ⎟     
  ────────────                     ────────────⎟     
     2⋅k⋅t₀                     2      k⋅t₀    ⎟     
⋅ℯ             + 4⋅m⋅t⋅{\Delta}v ⋅ℯ            ⎠     
──────────────────────────────────────────────── - M₂
                                           

In [32]:
System_solution = sympy.solve([zmoment, fmoment.dot(N.x), fmoment.dot(N.y), smoment], (A, v_0.dot(N.x), v_0.dot(N.y), t), dict=True)
Asol, vxsol, vysol, tsol = System_solution[0][A], System_solution[0][v_0.dot(N.x)], System_solution[0][v_0.dot(N.y)], System_solution[0][t]

In [33]:
display(sympy.Eq(A, Asol))
display(sympy.Eq(v_0.dot(N.x), vxsol))
display(sympy.Eq(v_0.dot(N.y), vysol))
display(sympy.Eq(t, tsol))
display(sympy.Eq(sigma_0,linearization))

                     2                    2                          2        
          m⋅{\Delta}v          m⋅{\Delta}v                m⋅{\Delta}v         
          ────────────         ────────────               ────────────        
             2⋅k⋅t₀                k⋅t₀                2     2⋅k⋅t₀           
    - M₂⋅ℯ             - 2⋅M₂⋅ℯ             + {\Delta}v ⋅ℯ             + 4⋅{\D
A = ──────────────────────────────────────────────────────────────────────────
                               ⎛                2               2             
                               ⎜   3⋅m⋅{\Delta}v     m⋅{\Delta}v       m⋅{\Del
                               ⎜   ──────────────    ────────────      ───────
                             2 ⎜       2⋅k⋅t₀           2⋅k⋅t₀             k⋅t
                    {\Delta}v ⋅⎝4⋅ℯ               + ℯ             + 4⋅ℯ       

                    2
         m⋅{\Delta}v 
         ────────────
      2      k⋅t₀    
elta}v ⋅ℯ            
────────────────────

                                                                     ⎛        
                                                                     ⎜   3⋅m⋅{
                                                                     ⎜   ─────
                                                                     ⎜       2
                                                        M_{1_x}⋅k⋅t₀⋅⎝4⋅ℯ     
v_{x_0} = ────────────────────────────────────────────────────────────────────
              ⎛                   2                      2                  2 
              ⎜      3⋅m⋅{\Delta}v          2⋅m⋅{\Delta}v        m⋅{\Delta}v  
              ⎜      ──────────────         ──────────────       ──────────── 
              ⎜          2⋅k⋅t₀                  k⋅t₀                k⋅t₀     
          2⋅m⋅⎝4⋅M₂⋅ℯ               + 4⋅M₂⋅ℯ               + M₂⋅ℯ             

        2               2                 2⎞                                  
\Delta}v     m⋅{\Delta}v       m⋅{\Delta}v ⎟       

                                                                     ⎛        
                                                                     ⎜   3⋅m⋅{
                                                                     ⎜   ─────
                                                                     ⎜       2
                                                        M_{1_y}⋅k⋅t₀⋅⎝4⋅ℯ     
v_{y_0} = ────────────────────────────────────────────────────────────────────
              ⎛                   2                      2                  2 
              ⎜      3⋅m⋅{\Delta}v          2⋅m⋅{\Delta}v        m⋅{\Delta}v  
              ⎜      ──────────────         ──────────────       ──────────── 
              ⎜          2⋅k⋅t₀                  k⋅t₀                k⋅t₀     
          2⋅m⋅⎝4⋅M₂⋅ℯ               + 4⋅M₂⋅ℯ               + M₂⋅ℯ             

        2               2                 2⎞                                  
\Delta}v     m⋅{\Delta}v       m⋅{\Delta}v ⎟       

           ⎛                    2                        2                    
           ⎜       9⋅m⋅{\Delta}v            7⋅m⋅{\Delta}v           5⋅m⋅{\Delt
           ⎜       ──────────────           ──────────────          ──────────
         2 ⎜           2⋅k⋅t₀                   2⋅k⋅t₀                  2⋅k⋅t₀
    -k⋅t₀ ⋅⎝64⋅M₂⋅ℯ               + 240⋅M₂⋅ℯ               + 60⋅M₂⋅ℯ          
t = ──────────────────────────────────────────────────────────────────────────
                               ⎛                    2                       2 
                               ⎜       9⋅m⋅{\Delta}v           7⋅m⋅{\Delta}v  
                               ⎜       ──────────────          ────────────── 
                             2 ⎜           2⋅k⋅t₀                  2⋅k⋅t₀     
                2⋅m⋅{\Delta}v ⋅⎝32⋅M₂⋅ℯ               + 80⋅M₂⋅ℯ               

   2                    2                        2                        2   
a}v        3⋅m⋅{\Delta}v            4⋅m⋅{\Delta}v  

                                                                     ⎛  2     
                                                                   m⋅⎝vₓ  + v_
            ⎛        ⎛  2      2⎞                               ⎞  ───────────
            ⎜    m⋅t⋅⎝vₓ  + v_y ⎠   m⋅vₓ⋅v_{x_0}   m⋅v_y⋅v_{y_0}⎟      2⋅k⋅t₀ 
\sigma₀ = A⋅⎜1 + ──────────────── - ──────────── - ─────────────⎟⋅ℯ           
            ⎜              2            k⋅t₀            k⋅t₀    ⎟             
            ⎝        2⋅k⋅t₀                                     ⎠             

 2⎞
y ⎠
───
   
   
   
   